# SData

In [1]:
#| default_exp sdata

In [2]:
#| hide
%load_ext autoreload
%autoreload 2

In [30]:
#| export
import torch, inspect, numpy as np, scipy.sparse as sp
from typing import Callable, Optional, Union, Dict
from torch.utils.data import DataLoader
from transformers import BatchEncoding

from xcai.core import Filterer, Info
from xcai.data import MainXCData, MetaXCData
from xcai.data import BaseXCDataset, MainXCDataset, MetaXCDataset, MetaXCDatasets
from xcai.data import BaseXCDataBlock
from xcai.data import _read_sparse_file
from xcai.graph.operations import *

from fastcore.utils import *
from fastcore.meta import *
from plum import dispatch

In [4]:
#| hide
from nbdev.showdoc import *
import nbdev; nbdev.nbdev_export()

## Data

In [5]:
dset_dir = '/Users/suchith720/Projects/data/(mapped)LF-WikiSeeAlsoTitles-320K'
data_cfg = {
    'info_column_names': ['identifier', 'input_text'],
    'use_tokenizer': True,
    'tokenizer': 'sentence-transformers/msmarco-distilbert-base-v4',
    'tokenization_column': 'input_text',
    'main_max_data_sequence_length': 32,
    'main_max_lbl_sequence_length': 32,
    'meta_max_sequence_length': 32,
    'padding': True,
    'return_tensors': 'pt',
}

In [21]:
train_cfg = {
    'data_lbl': f'{dset_dir}/trn_X_Y.txt',
    'data_info': f'{dset_dir}/raw_data/train.raw.txt',
    'lbl_info': f'{dset_dir}/raw_data/label.raw.txt',
    'data_lbl_filterer': f'{dset_dir}/filter_labels_train.txt',
}
train_data = MainXCData.from_file(**train_cfg, **data_cfg)

In [12]:
meta_cfg = {
    'prefix': 'hlk',
    'data_meta': f'{dset_dir}/hyper_link_trn_X_Y.txt',
    'lbl_meta': f'{dset_dir}/hyper_link_lbl_X_Y.txt',
    'meta_info': f'{dset_dir}/raw_data/hyper_link.raw.txt',
}
meta_data = MetaXCData.from_file(**meta_cfg, **data_cfg)

/home/scai/phd/aiz218323/.local/lib/python3.10/site-packages/xclib-0.97-py3.10-linux-x86_64.egg/xclib/data/data_utils.py:263: UserWarning: Header mis-match from inferred shape!
  warnings.warn("Header mis-match from inferred shape!")


In [13]:
#| export
def identity_collate_fn(batch): return BatchEncoding(batch)

## SDataset

### `SMainXCDataset`

In [35]:
#| export
class SMainXCDataset(MainXCDataset):

    def __init__(
        self,
        n_slbl_samples:Optional[int]=1,
        main_oversample:Optional[bool]=False,
        use_main_distribution:Optional[bool]=False,
        return_scores:Optional[bool]=False,
        **kwargs
    ):
        super().__init__(**kwargs)
        store_attr('n_slbl_samples,main_oversample,use_main_distribution,return_scores')
        
        self.data_lbl_scores = None
        if use_main_distribution or return_scores: self._store_scores()

    def _get_dataset(
        self, 
        data_info:Dict, 
        data_lbl:Optional[sp.csr_matrix]=None, 
        lbl_info:Optional[Dict]=None, 
        data_lbl_filterer:Optional[Union[sp.csr_matrix,np.array]]=None, 
        **kwargs
    ):
        n_lbl_samples = kwargs.get('n_lbl_samples') if 'n_lbl_samples' in kwargs else self.n_lbl_samples
        data_info_keys = kwargs.get('data_info_keys') if 'data_info_keys' in kwargs else self.data_info_keys
        lbl_info_keys = kwargs.get('lbl_info_keys') if 'lbl_info_keys' in kwargs else self.lbl_info_keys
        n_slbl_samples = kwargs.get('n_slbl_samples') if 'n_slbl_samples' in kwargs else self.n_slbl_samples
        main_oversample = kwargs.get('main_oversample') if 'main_oversample' in kwargs else self.main_oversample
        use_main_distribution = kwargs.get('use_main_distribution') if 'use_main_distribution' in kwargs else self.use_main_distribution
        return_scores = kwargs.get('return_scores') if 'return_scores' in kwargs else self.return_scores
        
        return SMainXCDataset(data_info=data_info, data_lbl=data_lbl, lbl_info=lbl_info, data_lbl_filterer=data_lbl_filterer, 
                              n_lbl_samples=n_lbl_samples, data_info_keys=data_info_keys, lbl_info_keys=lbl_info_keys, 
                              n_slbl_samples=n_slbl_samples, main_oversample=main_oversample, use_main_distribution=use_main_distribution, 
                              return_scores=return_scores)
        
    def _store_scores(self):
        if self.data_lbl is not None:
            if self.use_main_distribution:
                data_lbl = self.data_lbl / (self.data_lbl.sum(axis=1) + 1e-9)
                data_lbl = data_lbl.tocsr()
            else:
                data_lbl = self.data_lbl
            self.data_lbl_scores = [o.data.tolist() for o in data_lbl]
            
    def __getitems__(self, idxs:List):
        x = {'data_idx': torch.tensor(idxs, dtype=torch.int64)}
        x.update(self.get_info('data', idxs, self.data_info, self.data_info_keys))
        if self.data_lbl is not None:
            prefix = 'lbl2data'
            o = self.extract_items(prefix, self.curr_data_lbl, idxs, self.n_lbl_samples, self.n_slbl_samples, self.main_oversample, 
                                   self.lbl_info, self.lbl_info_keys, self.use_main_distribution, self.data_lbl_scores, 
                                   return_scores=self.return_scores)
            x.update(o)
        return x

    @classmethod
    @delegates(MainXCData.from_file)
    def from_file(
        cls, 
        n_lbl_samples:Optional[int]=None,
        data_info_keys:Optional[List]=None,
        lbl_info_keys:Optional[List]=None,
        n_slbl_samples:Optional[int]=1,
        main_oversample:Optional[bool]=False,
        use_main_distribution:Optional[bool]=False,
        return_scores:Optional[bool]=False,
        **kwargs
    ):
        return cls(**MainXCData.from_file(**kwargs), n_lbl_samples=n_lbl_samples, data_info_keys=data_info_keys, 
                   lbl_info_keys=lbl_info_keys, n_slbl_samples=n_slbl_samples, main_oversample=main_oversample, 
                   use_main_distribution=use_main_distribution, return_scores=return_scores)

    def _getitems(cls, idxs:List):
        return SMainXCDataset(
            data_info={k:v[idxs] if isinstance(v, torch.Tensor) or isinstance(v, np.ndarray) else [v[idx] for idx in idxs] for k,v in cls.data_info.items()}, 
            data_lbl=cls.data_lbl[idxs] if cls.data_lbl is not None else None, 
            lbl_info=cls.lbl_info, 
            data_lbl_filterer=Filterer.sample(cls.data_lbl_filterer, sz=cls.data_lbl.shape, idx=idxs) if cls.data_lbl_filterer is not None else None,
            n_lbl_samples=cls.n_lbl_samples,
            data_info_keys=cls.data_info_keys,
            lbl_info_keys=cls.lbl_info_keys,
            n_slbl_samples=cls.n_slbl_samples,
            main_oversample=cls.main_oversample,
            use_main_distribution=cls.use_main_distribution,
            return_scores=cls.return_scores,
        )
    

#### Example

In [ ]:
train_main = SMainXCDataset(**train_data, n_slbl_samples=2)

In [ ]:
train_main.__getitems__([100, 200])

{'data_idx': tensor([100, 200]),
 'data_identifier': ['Applet', 'Geography_of_Africa'],
 'data_input_text': ['Applet', 'Geography of Africa'],
 'data_input_ids': tensor([[  101,  6207,  2102,   102,     0,     0,     0,     0,     0,     0,
              0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
              0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
              0,     0],
         [  101, 10505,  1997,  3088,   102,     0,     0,     0,     0,     0,
              0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
              0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
              0,     0]]),
 'data_token_type_ids': tensor([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
          0, 0, 0, 0, 0, 0, 0, 0],
         [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
          0, 0, 0, 0, 0, 0, 0, 0]]),
 'data_attention_mask': tensor([[1, 1, 1, 1, 0, 

In [ ]:
train_main.oversample = True
train_main.n_slbl_samples = 3

In [ ]:
def func():
    import pdb; pdb.set_trace()
    train_main.__getitems__([1,2,3,4])
    

In [ ]:
train_dl = DataLoader(train_main, batch_size=10, collate_fn=identity_collate_fn)
batch = next(iter(train_dl))

In [ ]:
batch

{'data_idx': tensor([0, 1, 2, 3, 4, 5, 6, 7, 8, 9]),
 'data_identifier': ['Anarchism',
  'Autism',
  'Aristotle',
  'Academy_Awards',
  'International_Atomic_Time',
  'Altruism',
  'Allan_Dwan',
  'Anthropology',
  'Agricultural_science',
  'Animation'],
 'data_input_text': ['Anarchism',
  'Autism',
  'Aristotle',
  'Academy Awards',
  'International Atomic Time',
  'Altruism',
  'Allan Dwan',
  'Anthropology',
  'Agricultural science',
  'Animation'],
 'data_input_ids': tensor([[  101,  9617, 11140,  2964,   102,     0,     0,     0,     0,     0,
              0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
              0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
              0,     0],
         [  101, 19465,   102,     0,     0,     0,     0,     0,     0,     0,
              0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
              0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
              0,    

In [ ]:
tokz = AutoTokenizer.from_pretrained(data_cfg['tokenizer'])

In [ ]:
batch['plbl2data_data2ptr']

In [ ]:
tokz.batch_decode(batch['data_input_ids'])

### `SMetaXCDataset`

In [25]:
#| export
class SMetaXCDataset(MetaXCDataset):

    def __init__(
        self,
        n_sdata_meta_samples:Optional[int]=1,
        n_slbl_meta_samples:Optional[int]=1,
        meta_oversample:Optional[bool]=False,
        use_meta_distribution:Optional[bool]=False,
        meta_dropout_remove:Optional[float]=None,
        meta_dropout_replace:Optional[float]=None,
        return_scores:Optional[bool]=False,
        **kwargs
    ):
        super().__init__(**kwargs)
        store_attr('n_sdata_meta_samples,n_slbl_meta_samples,meta_oversample,use_meta_distribution')
        store_attr('meta_dropout_remove,meta_dropout_replace,return_scores')

        self.data_meta_scores, self.lbl_meta_scores = None, None
        if use_meta_distribution or return_scores: self._store_scores()

    def _store_scores(self):
        if self.data_meta is not None:
            if self.use_meta_distribution:
                data_meta = self.data_meta / (self.data_meta.sum(axis=1) + 1e-9)
                data_meta = data_meta.tocsr()
            else:
                data_meta = self.data_meta
            self.data_meta_scores = [o.data.tolist() for o in data_meta]
            
        if self.lbl_meta is not None:
            if self.use_meta_distribution:
                lbl_meta = self.lbl_meta / (self.lbl_meta.sum(axis=1) + 1e-9)
                lbl_meta = lbl_meta.tocsr()
            else:
                lbl_meta = self.lbl_meta
            self.lbl_meta_scores = [o.data.tolist() for o in lbl_meta]

    def _getitems(self, idxs:List):
        return SMetaXCDataset(prefix=self.prefix, data_meta=self.data_meta[idxs], lbl_meta=self.lbl_meta, meta_info=self.meta_info, 
                              n_data_meta_samples=self.n_data_meta_samples, n_lbl_meta_samples=self.n_lbl_meta_samples, 
                              meta_info_keys=self.meta_info_keys, n_sdata_meta_samples=self.n_sdata_meta_samples, 
                              n_slbl_meta_samples=self.n_slbl_meta_samples, meta_oversample=self.meta_oversample, 
                              use_meta_distribution=self.use_meta_distribution, meta_dropout_remove=self.meta_dropout_remove, 
                              meta_dropout_replace=self.meta_dropout_replace, return_scores=self.return_scores)

    def _sample_meta_items(self, idxs:List):
        assert max(idxs) < self.n_meta, f"indices should be less than {self.n_meta}"
        meta_info = {k: [v[i] for i in idxs] for k,v in self.meta_info.items()}
        return SMetaXCDataset(prefix=self.prefix, data_meta=self.data_meta[:, idxs], lbl_meta=self.lbl_meta[:, idxs], meta_info=meta_info, 
                              n_data_meta_samples=self.n_data_meta_samples, n_lbl_meta_samples=self.n_lbl_meta_samples,
                              meta_info_keys=self.meta_info_keys, n_sdata_meta_samples=self.n_sdata_meta_samples, 
                              n_slbl_meta_samples=self.n_slbl_meta_samples, meta_oversample=self.meta_oversample, 
                              use_meta_distribution=self.use_meta_distribution, meta_dropout_remove=self.meta_dropout_remove, 
                              meta_dropout_replace=self.meta_dropout_replace, return_scores=self.return_scores)
        
    @classmethod
    @delegates(MetaXCData.from_file)
    def from_file(
        cls, 
        n_data_meta_samples:Optional[int]=None, 
        n_lbl_meta_samples:Optional[int]=None, 
        meta_info_keys:Optional[List]=None,
        n_sdata_meta_samples:Optional[int]=1,
        n_slbl_meta_samples:Optional[int]=1,
        meta_oversample:Optional[bool]=False,
        use_meta_distribution:Optional[bool]=False,
        meta_dropout_remove:Optional[float]=None,
        meta_dropout_replace:Optional[float]=None,
        return_scores:Optional[bool]=False,
        **kwargs
    ):
        return cls(**MetaXCData.from_file(**kwargs), n_data_meta_samples=n_data_meta_samples, n_lbl_meta_samples=n_lbl_meta_samples, 
                   meta_info_keys=meta_info_keys, n_sdata_meta_samples=n_sdata_meta_samples, n_slbl_meta_samples=n_slbl_meta_samples,
                   meta_oversample=meta_oversample, use_meta_distribution=use_meta_distribution, meta_dropout_remove=meta_dropout_remove, 
                   meta_dropout_replace=meta_dropout_replace, return_scores=return_scores)

    def get_data_meta(self, idxs:List):
        x, prefix = dict(), f'{self.prefix}2data'
        o = self.extract_items(prefix, self.curr_data_meta, idxs, self.n_data_meta_samples, self.n_sdata_meta_samples, self.meta_oversample, 
                               self.meta_info, self.meta_info_keys, self.use_meta_distribution, self.data_meta_scores, 
                               dropout_remove=self.meta_dropout_remove, dropout_replace=self.meta_dropout_replace, 
                               return_scores=self.return_scores)
        x.update(o)
        return x
        
    def get_lbl_meta(self, idxs:List):
        x, prefix = dict(), f'{self.prefix}2lbl'
        o = self.extract_items(prefix, self.curr_lbl_meta, idxs, self.n_lbl_meta_samples, self.n_slbl_meta_samples, self.meta_oversample, 
                               self.meta_info, self.meta_info_keys, self.use_meta_distribution, self.lbl_meta_scores, 
                               dropout_remove=self.meta_dropout_remove, dropout_replace=self.meta_dropout_replace, 
                               return_scores=self.return_scores)
        x.update(o)
        return x

    def _verify_inputs(cls):
        cls.n_data,cls.n_meta = cls.data_meta.shape[0],cls.data_meta.shape[1]
        
        if cls.lbl_meta is not None:
            cls.n_lbl = cls.lbl_meta.shape[0]
            if cls.lbl_meta.shape[1] != cls.n_meta:
                raise ValueError(f'`lbl_meta`({cls.lbl_meta.shape[1]}) should have same number of columns as `data_meta`({cls.n_meta}).')
    
        if cls.meta_info is not None:
            n_meta = cls._verify_info(cls.meta_info)
            if n_meta != cls.n_meta:
                raise ValueError(f'`meta_info`({n_meta}) should have same number of entries as number of columns of `data_meta`({cls.n_meta})')
            if cls.meta_info_keys is None: cls.meta_info_keys = list(cls.meta_info.keys())
        

#### Example

In [ ]:
train_meta = SMetaXCDataset(**meta_data, n_sdata_meta_samples=2, n_slbl_meta_samples=2)

In [ ]:
train_meta.meta_oversample = True
train_meta.n_sdata_meta_samples = 3
train_meta.n_slbl_meta_samples = 3

In [ ]:
train_meta.get_data_meta([100, 200])

In [ ]:
train_meta.get_lbl_meta([101, 201])

### `SXCDataset`

In [37]:
#| export
class SXCDataset(BaseXCDataset):

    def __init__(self, data:SMainXCDataset, **kwargs):
        super().__init__()
        self.data, self.meta = data, MetaXCDatasets({k:kwargs[k] for k in self.get_meta_args(**kwargs) if isinstance(kwargs[k], SMetaXCDataset)})
        self._verify_inputs()

    def _getitems(self, idxs:List):
        return SXCDataset(self.data._getitems(idxs), **{k:meta._getitems(idxs) for k,meta in self.meta.items()})
        
    @classmethod
    @delegates(SMainXCDataset.from_file)
    def from_file(cls, **kwargs):
        data = SMainXCDataset.from_file(**kwargs)
        meta_kwargs = {o:kwargs.pop(o) for o in cls.get_meta_args(**kwargs)}
        meta = {k:SMetaXCDataset.from_file(**v, **kwargs) for k,v in meta_kwargs.items()}
        return cls(data, **meta)

    @staticmethod
    def get_meta_args(**kwargs):
        return [k for k in kwargs if re.match(r'.*_meta$', k)]

    def _verify_inputs(self):
        self.n_data, self.n_lbl = self.data.n_data, self.data.n_lbl
        if len(self.meta):
            self.n_meta = len(self.meta)
            for meta in self.meta.values():
                if meta.n_data != self.n_data: 
                    raise ValueError(f'`meta`({meta.n_data}) and `data`({self.n_data}) should have the same number of datapoints.')
                if self.n_lbl is not None and meta.n_lbl != self.n_lbl: 
                    raise ValueError(f'`meta`({meta.n_lbl}) and `data`({self.n_lbl}) should have the same number of labels.')

    def __getitems__(self, idxs:List):
        x = self.data.__getitems__(idxs)
        if self.n_meta:
            for meta in self.meta.values():
                x.update(meta.get_data_meta(idxs))
                if self.n_lbl:
                    z = meta.get_lbl_meta(x['lbl2data_idx'])
                    z[f'{meta.prefix}2lbl_data2ptr'] = torch.tensor([o.sum() for o in z[f'{meta.prefix}2lbl_lbl2ptr'].split_with_sizes(x[f'lbl2data_data2ptr'].tolist())])
                    z[f'p{meta.prefix}2lbl_data2ptr'] = torch.tensor([o.sum() for o in z[f'p{meta.prefix}2lbl_lbl2ptr'].split_with_sizes(x[f'lbl2data_data2ptr'].tolist())])
                    x.update(z)
        return x

    @property
    def lbl_info(self): return self.data.lbl_info

    @property
    def lbl_dset(self): return SMainXCDataset(data_info=self.data.lbl_info, n_lbl_samples=self.data.n_lbl_samples, 
                                               data_info_keys=self.data.data_info_keys, lbl_info_keys=self.data.lbl_info_keys, 
                                               n_slbl_samples=self.data.n_slbl_samples, main_oversample=self.data.main_oversample, 
                                               use_main_distribution=self.data.use_main_distribution, return_scores=self.data.return_scores)
        
    def lbl_meta_dset(self, meta_name):
        m = self.meta[f'{meta_name}_meta']
        return SMetaXCDataset(prefix=m.prefix, data_meta=m.lbl_meta, lbl_meta=m.lbl_meta, meta_info=m.meta_info, 
                              n_data_meta_samples=m.n_data_meta_samples, n_lbl_meta_samples=m.n_lbl_meta_samples, 
                              meta_info_keys=m.meta_info_keys, n_sdata_meta_samples=m.n_sdata_meta_samples, 
                              n_slbl_meta_samples=m.n_slbl_meta_samples, meta_oversample=m.meta_oversample, 
                              use_meta_distribution=m.use_meta_distribution, meta_dropout_remove=m.meta_dropout_remove, 
                              meta_dropout_replace=m.meta_dropout_replace, return_scores=m.return_scores)
        
    @property
    def data_info(self): return self.data.data_info

    @property
    def data_dset(self): return SMainXCDataset(data_info=self.data.data_info, n_lbl_samples=self.data.n_lbl_samples, 
                                               data_info_keys=self.data.data_info_keys, lbl_info_keys=self.data.lbl_info_keys, 
                                               n_slbl_samples=self.data.n_slbl_samples, main_oversample=self.data.main_oversample, 
                                               use_main_distribution=self.data.use_main_distribution, return_scores=self.data.return_scores)
        
    def data_meta_dset(self, meta_name):
        return self.meta[f'{meta_name}_meta']
        
    def one_batch(self, bsz:Optional[int]=10, seed:Optional[int]=None):
        if seed is not None: torch.manual_seed(seed)
        idxs = list(torch.randperm(len(self)).numpy())[:bsz]
        return [self[idx] for idx in idxs]

    def get_one_hop_metadata(self, batch_size:Optional[int]=1024, thresh:Optional[int]=10, topk:Optional[int]=10, **kwargs):
        data_lbl = Graph.threshold_on_degree(self.data.data_lbl, thresh=thresh)
        data_meta, lbl_meta = Graph.one_hop_matrix(data_lbl, batch_size=batch_size, topk=topk, do_normalize=True)
        self.meta['ohm_meta'] = SMetaXCDataset(prefix='ohm', data_meta=data_meta, lbl_meta=lbl_meta, 
                                               meta_info=self.data.lbl_info, **kwargs)

    def get_random_walk_metadata(self, batch_size:Optional[int]=1024, walk_to:Optional[int]=100, prob_reset:Optional[float]=0.8, 
                                 topk_thresh:Optional[int]=10, degree_thresh=20, **kwargs):
        data_meta = perform_random_walk(data_lbl, batch_size=batch_size, walk_to=walk_to, prob_reset=prob_reset, 
                                        n_hops=1, thresh=degree_thresh, topk=topk_thresh, do_normalize=True)
        lbl_meta = perform_random_walk(data_lbl.transpose().tocsr(), batch_size=batch_size, walk_to=walk_to, 
                                       prob_reset=prob_reset, n_hops=2, thresh=degree_thresh, topk=topk_thresh, do_normalize=True)
        self.meta['rnw_meta'] = SMetaXCDataset(prefix='rnw', data_meta=data_meta, lbl_meta=lbl_meta,
                                               meta_info=self.data.lbl_info, **kwargs)

    def get_random_walk_with_matrices_metadata(self, meta_name:str, batch_size:Optional[int]=1024, walk_to:Optional[int]=100, 
                                               prob_reset:Optional[float]=0.8, topk_thresh:Optional[int]=10, data_degree_thresh=20, 
                                               lbl_degree_thresh=20, **kwargs):
        if f'{meta_name}_meta' not in self.meta: raise ValueError(f'Invalid metadata: {meta_name}')
        data_meta, lbl_meta = self.meta[f'{meta_name}_meta'].data_meta, self.meta[f'{meta_name}_meta'].lbl_meta
        data_rnw = perform_random_walk_with_matrices(data_meta, lbl_meta, batch_size=batch_size, walk_to=walk_to, prob_reset=prob_reset, 
                                                     n_hops=2, data_thresh=data_degree_thresh, lbl_thresh=lbl_degree_thresh, 
                                                     topk=topk_thresh, do_normalize=True)
        lbl_rnw = perform_random_walk_with_matrices(lbl_meta, data_meta, batch_size=batch_size, walk_to=walk_to, prob_reset=prob_reset, 
                                                    n_hops=3, data_thresh=data_degree_thresh, lbl_thresh=lbl_degree_thresh, 
                                                    topk=topk_thresh, do_normalize=True)
        self.meta['rnw_meta'] = SMetaXCDataset(prefix='rnw', data_meta=data_rnw, lbl_meta=lbl_rnw, meta_info=self.data.lbl_info, **kwargs)
        
    @staticmethod
    def combine_info(info_1:Dict, info_2:Dict, pad_token:int=0):
        comb_info = dict()
        for k,v in info_1.items():
            if isinstance(v, tuple) or isinstance(v, list): comb_info[k] = v + info_2[k]
            elif isinstance(v, torch.Tensor):
                n_data = v.shape[0] + info_2[k].shape[0]
                seq_len = max(v.shape[1], info_2[k].shape[1]) 
                
                if k == 'input_ids': 
                    info = torch.full((n_data, seq_len), pad_token, dtype=v.dtype)
                elif k == 'attention_mask': 
                    info = torch.full((n_data, seq_len), 0, dtype=v.dtype)
                    
                info[:v.shape[0], :v.shape[1]] = v
                info[v.shape[0]:, :info_2[k].shape[1]] = info_2[k]
                
                comb_info[k] = info
        return comb_info

    def _get_main_dataset(
        self,
        data_info:Dict, 
        data_lbl:Optional[sp.csr_matrix]=None, 
        lbl_info:Optional[Dict]=None, 
        data_lbl_filterer:Optional[Union[sp.csr_matrix,np.array]]=None, 
        **kwargs
    ):
        dset = self.data._get_dataset(data_info, data_lbl, lbl_info, data_lbl_filterer, **kwargs)
        return SXCDataset(dset)
        
    def combine_lbl_and_meta(self, meta_name:str, pad_token:int=0, p_data=0.5, **kwargs): 
        if f'{meta_name}_meta' not in self.meta: raise ValueError(f'Invalid metadata: {meta_name}')
            
        data_lbl = self.data.data_lbl
        data_lbl = data_lbl.multiply(1/(data_lbl.getnnz(axis=1).reshape(-1, 1) + 1e-9))
        data_lbl = data_lbl.tocsr() * p_data
        
        data_meta = self.meta[f'{meta_name}_meta'].data_meta
        data_meta = data_meta.multiply(1/(data_meta.getnnz(axis=1).reshape(-1, 1) + 1e-9))
        data_meta = data_meta.tocsr() * (1 - p_data)
        
        lbl_info = self.data.lbl_info
        meta_info = self.meta[f'{meta_name}_meta'].meta_info

        comb_info = self.combine_info(lbl_info, meta_info, pad_token)
        
        return self._get_main_dataset(self.data.data_info, sp.hstack([data_lbl, data_meta]), comb_info, 
                                      self.data.data_lbl_filterer, **kwargs)

    def combine_data_and_meta(self, meta_name:str, pad_token:int=0, **kwargs):
        if f'{meta_name}_meta' not in self.meta: raise ValueError(f'Invalid metadata: {meta_name}')
        
        data_lbl, meta_lbl = self.data.data_lbl, self.meta[f'{meta_name}_meta'].lbl_meta.transpose().tocsr()
        assert data_lbl.shape[1] == meta_lbl.shape[1], f"Incompatible metadata shape: {meta_lbl.shape}"

        data_info = self.data.data_info
        meta_info = self.meta[f'{meta_name}_meta'].meta_info
        comb_info = self.combine_info(data_info, meta_info, pad_token)

        dset = self._get_main_dataset(comb_info, sp.vstack([data_lbl, meta_lbl]), self.data.lbl_info, 
                                      self.data.data_lbl_filterer, **kwargs)
        valid_idx = np.where(dset.data.data_lbl.getnnz(axis=1) > 0)[0]
        return dset._getitems(valid_idx)

    @staticmethod
    def get_combined_data_and_meta(dset, meta_lbl:sp.csr_matrix, meta_info:Dict, pad_token:int=0, **kwargs):    
        data_lbl = dset.data.data_lbl
        assert data_lbl.shape[1] == meta_lbl.shape[1], f"Incompatible metadata shape: {meta_lbl.shape}"
        
        data_info = dset.data.data_info
        comb_info = dset.combine_info(data_info, meta_info, pad_token)
        
        dset = dset._get_main_dataset(comb_info, sp.vstack([data_lbl, meta_lbl]), dset.data.lbl_info, 
                                      dset.data.data_lbl_filterer, **kwargs)
        valid_idx = np.where(dset.data.data_lbl.getnnz(axis=1) > 0)[0]
        return dset._getitems(valid_idx)
        
        

#### Example

In [ ]:
train_dset = SXCDataset(train_main, hlk_meta=train_meta)

In [ ]:
bb = train_dset.__getitems__([100, 200, 500])

In [ ]:
train_dl = DataLoader(train_dset, batch_size=10, collate_fn=identity_collate_fn)
batch = next(iter(train_dl))

In [ ]:
list(batch)

['data_idx',
 'data_identifier',
 'data_input_text',
 'data_input_ids',
 'data_token_type_ids',
 'data_attention_mask',
 'plbl2data_idx',
 'plbl2data_data2ptr',
 'lbl2data_idx',
 'lbl2data_data2ptr',
 'lbl2data_identifier',
 'lbl2data_input_text',
 'lbl2data_input_ids',
 'lbl2data_token_type_ids',
 'lbl2data_attention_mask',
 'phlk2data_idx',
 'phlk2data_data2ptr',
 'hlk2data_idx',
 'hlk2data_data2ptr',
 'hlk2data_identifier',
 'hlk2data_input_text',
 'hlk2data_input_ids',
 'hlk2data_token_type_ids',
 'hlk2data_attention_mask',
 'phlk2lbl_idx',
 'phlk2lbl_lbl2ptr',
 'hlk2lbl_idx',
 'hlk2lbl_lbl2ptr',
 'hlk2lbl_identifier',
 'hlk2lbl_input_text',
 'hlk2lbl_input_ids',
 'hlk2lbl_token_type_ids',
 'hlk2lbl_attention_mask',
 'hlk2lbl_data2ptr',
 'phlk2lbl_data2ptr']

### `SBaseXCDataBlock`

In [32]:
#| export
class SBaseXCDataBlock(BaseXCDataBlock):

    @delegates(DataLoader.__init__)
    def __init__(
        self, 
        dset:SXCDataset, 
        collate_fn:Optional[Callable]=identity_collate_fn,
        **kwargs
    ):
        self.dset, self.dl_kwargs, self.collate_fn = dset, self._get_dl_kwargs(**kwargs), collate_fn
        self.dl = DataLoader(dset, collate_fn=collate_fn, **self.dl_kwargs) if collate_fn is not None else None

    def _get_dl_kwargs(self, **kwargs):
        dl_params = inspect.signature(DataLoader.__init__).parameters
        return {k:v for k,v in kwargs.items() if k in dl_params}

    @classmethod
    @delegates(SXCDataset.from_file)
    def from_file(cls, collate_fn:Callable=identity_collate_fn, **kwargs):
        return SBaseXCDataBlock(SXCDataset.from_file(**kwargs), collate_fn, **kwargs)

    def __len__(self):
        return len(self.dset)    
    
    def _getitems(self, idxs:List):
        return SBaseXCDataBlock(self.dset._getitems(idxs), collate_fn=self.collate_fn, **self.dl_kwargs)

    @property
    def bsz(self): return self.dl.batch_size

    @bsz.setter
    def bsz(self, v):
        self.dl_kwargs['batch_size'] = v
        self.dl = DataLoader(self.dset, collate_fn=self.collate_fn, **self.dl_kwargs) if self.collate_fn is not None else None

    @property
    def data_lbl_filterer(self): return self.dset.data.data_lbl_filterer

    @data_lbl_filterer.setter
    def data_lbl_filterer(self, val): self.dset.data.data_lbl_filterer = val

    @dispatch
    def one_batch(self):
        return next(iter(self.dl))

    @dispatch
    def one_batch(self, bsz:int):
        self.dl_kwargs['batch_size'] = bsz
        self.dl = DataLoader(self.dset, collate_fn=self.collate_fn, **self.dl_kwargs) if self.collate_fn is not None else None
        return next(iter(self.dl))
        
    def filterer(cls, train:'SBaseXCDataBlock', valid:'SBaseXCDataBlock', fld:Optional[str]='identifier'):
        train_info, valid_info, lbl_info = train.dset.data.data_info, valid.dset.data.data_info, train.dset.data.lbl_info
        if fld not in train_info: raise ValueError(f'`{fld}` not in `data_info`')
            
        train.data_lbl_filterer, valid_filterer = Filterer.generate(train_info[fld], valid_info[fld], lbl_info[fld], 
                                                                    train.dset.data.data_lbl, valid.dset.data.data_lbl)
        _, valid_filterer, idx = Filterer.prune(valid.dset.data.data_lbl, valid_filterer)
        
        valid = valid._getitems(idx)
        valid.data_lbl_filterer = valid_filterer
        
        return train, valid
        
    def splitter(cls, valid_pct:Optional[float]=0.2, seed=None):
        if seed is not None: torch.manual_seed(seed)
        rnd_idx = list(torch.randperm(len(cls)).numpy())
        cut = int(valid_pct * len(cls))
        train, valid = cls._getitems(rnd_idx[cut:]), cls._getitems(rnd_idx[:cut])
        if cls.data_lbl_filterer is None: return train, valid
        else: return cls.filterer(train, valid)

    def sample(cls, pct:Optional[float]=0.2, n:Optional[int]=None, seed=None):
        if seed is not None: torch.manual_seed(seed)
        rnd_idx = list(torch.randperm(len(cls)).numpy())
        cut = int(pct * len(cls)) if n is None else max(1, n)
        return cls._getitems(rnd_idx[:cut])
        

#### Example

In [ ]:
train_block = SBaseXCDataBlock(train_dset, batch_size=2)

### `SXCDataBlock`

In [36]:
#| export
class SXCDataBlock:

    def __init__(self, train:SBaseXCDataBlock=None, valid:SBaseXCDataBlock=None, test:SBaseXCDataBlock=None):
        self.train, self.valid, self.test = train, valid, test

    @staticmethod
    def load_cfg(fname):
        with open(fname, 'r') as f: return json.load(f)

    def sample_info(self, info, idx):
        return {k: v[idx] if isinstance(v, torch.Tensor) or isinstance(v, np.ndarray) else [v[i] for i in idx] for k,v in info.items()}

    def linker_dset(self, meta_name:str, remove_empty:Optional[bool]=True):
        if meta_name not in self.train.dset.meta:
            raise ValueError(f'Invalid metadata: {meta_name}')

        train_meta = self.train.dset.meta[meta_name].data_meta
        if remove_empty:
            train_idx = np.where(train_meta.getnnz(axis=1) > 0)[0]
            meta_idx = np.where(train_meta.getnnz(axis=0) > 0)[0]
            train_meta = train_meta[train_idx][:, meta_idx].tocsr()
        
        train_info = self.train.dset.data.data_info
        meta_info = self.train.dset.meta[meta_name].meta_info

        if remove_empty:
            train_info = self.sample_info(train_info, train_idx)
            meta_info = self.sample_info(meta_info, meta_idx)

        collate_fn = self.train.collate_fn
        train_dset = SBaseXCDataBlock(SXCDataset(SMainXCDataset(data_info=train_info, data_lbl=train_meta, lbl_info=meta_info)), 
                                      collate_fn=collate_fn)
        
        if self.test is not None:
            test_meta = self.test.dset.meta[meta_name].data_meta
            if remove_empty:
                test_meta = test_meta[:, meta_idx].tocsr()
                test_idx = np.where(test_meta.getnnz(axis=1) > 0)[0]
                test_meta = test_meta[test_idx].tocsr()
    
            test_info = self.test.dset.data.data_info
            if remove_empty: test_info = self.sample_info(test_info, test_idx)
    
            test_dset = SBaseXCDataBlock(SXCDataset(SMainXCDataset(data_info=test_info, data_lbl=test_meta, lbl_info=meta_info)), 
                                         collate_fn=collate_fn)
            return SXCDataBlock(train=train_dset, test=test_dset)
        
        return SXCDataBlock(train=train_dset)

    @staticmethod
    def inference_dset(data_info:Dict, data_lbl:sp.csr_matrix, lbl_info:Dict, data_lbl_filterer, **kwargs):
        x_idx = np.where(data_lbl.getnnz(axis=1) == 0)[0].reshape(-1,1)
        y_idx = np.zeros((len(x_idx),1), dtype=np.int64)
        data_lbl[x_idx, y_idx] = 1
        data_lbl_filterer = np.hstack([x_idx, y_idx]) if data_lbl_filterer is None else np.vstack([np.hstack([x_idx, y_idx]), data_lbl_filterer])
    
        pred_dset = SXCDataset(SMainXCDataset(data_info=data_info, data_lbl=data_lbl, lbl_info=lbl_info,
                                              data_lbl_filterer=data_lbl_filterer, **kwargs))
        return pred_dset

    @property
    def lbl_info(self): return self.test.dset.data.lbl_info if self.train is None else self.train.dset.data.lbl_info

    @property
    def lbl_dset(self): return SMainXCDataset(data_info=self.train.dset.data.lbl_info)

    @property
    def n_lbl(self): return self.test.dset.n_lbl if self.train is None else self.train.dset.n_lbl

    @property
    def collator(self): return self.test.collate_fn if self.train is None else self.train.collate_fn
        
    @classmethod
    def from_cfg(
        cls, 
        cfg:Union[str,Dict],
        collate_fn:Optional[Callable]=identity_collate_fn,
        valid_pct:Optional[float]=0.2,
        seed=None,
        **kwargs,
    ):
        if isinstance(cfg, str): cfg = cls.load_cfg(cfg)

        blocks = dict()
        for o in ['train', 'valid', 'test']:
            if o in cfg['path']:
                params = cfg['parameters'].copy()
                params.update(kwargs)
                if o != 'train': 
                    params['meta_dropout_remove'], params['meta_dropout_replace'] = None, None
                blocks[o] = SBaseXCDataBlock.from_file(**cfg['path'][o], **params, collate_fn=collate_fn)
                
        return cls(**blocks)
        

#### Example

In [ ]:
from xcai.config import WIKISEEALSOTITLES

In [ ]:
block = SXCDataBlock(train=train_block)

In [ ]:
linker_dset = block.linker_dset('hlk_meta')

In [ ]:
batch = next(iter(block.train.dl))

In [ ]:
batch.keys()

In [ ]:
config = WIKISEEALSOTITLES('/home/scai/phd/aiz218323/Projects/XC/data')['data_lnk']

In [ ]:
print(config['parameters'])

{'transform_type': 'xc', 'smp_features': [('lbl2data', 1, 2), ('hlk2data', 1, 1), ('hlk2lbl2data', 2, 1)], 'pad_token': 0, 'oversample': False, 'sampling_features': [('lbl2data', 2), ('hlk2data', 1), ('hlk2lbl2data', 1)], 'num_labels': 1, 'num_metadata': 1, 'metadata_name': None, 'info_column_names': ['identifier', 'input_text'], 'use_tokenizer': True, 'tokenizer': 'bert-base-cased', 'tokenization_column': 'input_text', 'max_sequence_length': 32, 'padding': False, 'return_tensors': None, 'sep': '->', 'prompt_func': None, 'pad_side': 'right', 'drop': True, 'ret_t': True, 'in_place': True, 'collapse': True, 'device': 'cpu', 'inp': 'data', 'targ': 'lbl2data', 'ptr': 'lbl2data_data2ptr', 'n_lbl_samples': None, 'data_info_keys': None, 'lbl_info_keys': None, 'n_slbl_samples': 1, 'main_oversample': False, 'n_data_meta_samples': 1, 'n_lbl_meta_samples': 1, 'meta_info_keys': None, 'meta_oversample': False}


In [ ]:
params = {'return_tensors':'pt', 'padding':True}

for k,v in params.items():
    config['parameters'][k] = v

In [ ]:
block = SXCDataBlock.from_cfg(config)

In [ ]:
batch = block.train.dset.__getitems__([100, 200])

In [ ]:
batch

{'data_idx': tensor([100, 200]),
 'data_identifier': ['Applet', 'Geography_of_Africa'],
 'data_input_text': ['Applet', 'Geography of Africa'],
 'data_input_ids': tensor([[  101,  7302,  1204,   102,     0,     0,     0,     0,     0,     0,
              0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
              0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
              0,     0],
         [  101, 20678,  1104,  2201,   102,     0,     0,     0,     0,     0,
              0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
              0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
              0,     0]]),
 'data_token_type_ids': tensor([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
          0, 0, 0, 0, 0, 0, 0, 0],
         [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
          0, 0, 0, 0, 0, 0, 0, 0]]),
 'data_attention_mask': tensor([[1, 1, 1, 1, 0, 